3-1. 네이버 책 검색 API 호출하기

In [1]:
# 캡차 키 발급 요청 (실행시 결과 키 값:scwWeA9fEZRO0S5v)
import urllib.request

code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
request = urllib.request.Request(url)
#요청 헤더 설정 (CLIENT_ID,CLIENT_SECRET = env 파일에 작성한 것 읽어오기) 
request.add_header("X-Naver-Client-Id",CLIENT_ID) 
request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)

response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

NameError: name 'CLIENT_ID' is not defined

In [ ]:
import requests
from pprint import pprint #(from pprint(모듈이름-pprint.py) import pprint(pprint라는 모듈 안에 있는 pprint라는 함수이름) )

headers = {
    'X-Naver-Client-Id': CLIENT_ID,
    'X-Naver-Client-Secret': CLIENT_SECRET,
}

def search_books(query):   
    # query string 문자열을 dict 선언
    payload = {
    'query':  query, #'파이썬',
    'display': 100,
    'sort': 'sim'
    }

url = 'https://openapi.naver.com/v1/search/book.json'